In [4]:
from plotly import graph_objects as go

import pandas as pd
import pandas_ta as ta
import yfinance as yf
import numpy as np

from plotly.subplots import make_subplots
import os
from ta.volatility import BollingerBands
from dash import Dash, dcc, html, Input, Output

In [16]:
def load_data(symbol, interval):
    data = yf.download(tickers=symbol, period="1d", interval=interval)
    df = pd.DataFrame(data)
    df.dropna(inplace=True)
    df = df.iloc[10:190]
    df['Date'] = pd.to_datetime(df.index)
    #  Create some indicators
    df['EMA_9'] = ta.ema(df['Close'], length=9)
    df['EMA_21'] = ta.ema(df['Close'], length=21)
    # Initialize Bollinger Bands Indicator
    indicator_bb = BollingerBands(close=df['Close'], window=20, window_dev=2)
    # Add Bollinger Bands features
    df['BB_HIGH'] = indicator_bb.bollinger_hband()
    df['BB_LOW'] = indicator_bb.bollinger_lband()
    return df



In [22]:
symbol = "GOOGL"
interval = "1m"
df = load_data(symbol, interval)

if 'EMA_22' in df.columns:
    print("yes")


[*********************100%***********************]  1 of 1 completed


In [7]:
print(df.columns)
print(df['Date'])

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Date', 'EMA_9',
       'EMA_21', 'BB_HIGH', 'BB_LOW'],
      dtype='object')
Datetime
2023-03-03 09:40:00-05:00   2023-03-03 09:40:00-05:00
2023-03-03 09:41:00-05:00   2023-03-03 09:41:00-05:00
2023-03-03 09:42:00-05:00   2023-03-03 09:42:00-05:00
2023-03-03 09:43:00-05:00   2023-03-03 09:43:00-05:00
2023-03-03 09:44:00-05:00   2023-03-03 09:44:00-05:00
                                       ...           
2023-03-03 12:35:00-05:00   2023-03-03 12:35:00-05:00
2023-03-03 12:36:00-05:00   2023-03-03 12:36:00-05:00
2023-03-03 12:37:00-05:00   2023-03-03 12:37:00-05:00
2023-03-03 12:38:00-05:00   2023-03-03 12:38:00-05:00
2023-03-03 12:39:00-05:00   2023-03-03 12:39:00-05:00
Name: Date, Length: 180, dtype: datetime64[ns, America/New_York]


In [8]:
df2 = pd.read_parquet('DATA/btcusdt/2021-01-01.parquet', engine='fastparquet')
print(df2.columns)
print(df2)

Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')
                         open      high       low     close     volume
timestamp                                                             
2021-01-01 00:01:00  28922.42  28961.66  28913.12  28961.66  27.234814
2021-01-01 00:02:00  28961.67  29017.50  28961.01  29009.91  58.477501
2021-01-01 00:03:00  29009.54  29016.71  28973.58  28989.30  42.470329
2021-01-01 00:04:00  28989.68  28999.85  28972.33  28982.69  30.360677
2021-01-01 00:05:00  28982.67  28995.93  28971.80  28975.65  24.124339
...                       ...       ...       ...       ...        ...
2021-01-01 23:56:00  29300.39  29315.00  29300.38  29309.60  13.034801
2021-01-01 23:57:00  29309.59  29317.22  29303.10  29317.22  18.114122
2021-01-01 23:58:00  29317.22  29319.26  29311.00  29319.20  11.583345
2021-01-01 23:59:00  29319.20  29328.59  29314.00  29322.54  34.533058
2021-01-02 00:00:00  29322.77  29338.89  29322.18  29331.69  29.966698

[1440 rows

In [14]:
from pathlib import Path
import pandas as pd

data_dir = Path('DATA/btcusdt')

df = pd.concat(
    pd.read_parquet(parquet_file, engine='fastparquet')
    for parquet_file in data_dir.glob('*.parquet')
)
df['date'] = pd.to_datetime(df.index)
df.set_index("date", inplace=True)

In [15]:
print(len(df))
df_2 = df.resample("1m").agg(ta.CANGLE_AGG)
print(df.iloc[0:5])
print(df_2.iloc[0:5])

525713
                         open      high       low     close      volume
date                                                                   
2021-08-03 00:01:00  39146.86  39186.09  39108.71  39140.00  113.693042
2021-08-03 00:02:00  39140.00  39169.85  39019.81  39091.44   83.994643
2021-08-03 00:03:00  39093.74  39125.14  39028.35  39039.28  257.948455
2021-08-03 00:04:00  39039.27  39054.34  38899.98  39011.33  186.984536
2021-08-03 00:05:00  39011.32  39018.51  38915.51  38956.23   96.994671
                open     high       low     close        volume
date                                                           
2021-01-31  28922.42  41950.0  28130.00  33087.33  3.436445e+06
2021-02-28  33085.89  58352.8  32296.16  45063.42  2.516925e+06
2021-03-31  45061.74  61844.0  44950.53  58731.37  2.097825e+06
2021-04-30  58731.37  64854.0  46930.00  57717.55  1.993425e+06
2021-05-31  57717.55  59500.0  30000.00  37307.15  3.535809e+06


In [11]:
def load_data_2(symbol):
    data_dir = Path('DATA/' + symbol)
    df = pd.concat(
        pd.read_parquet(parquet_file, engine='fastparquet')
        for parquet_file in data_dir.glob('*.parquet')
    )
    df.dropna(inplace=True)
    df = df.iloc[10:190]
    df['Date'] = pd.to_datetime(df.index)
    #  Create some indicators
    df['SMA_9'] = ta.sma(df['close'], length=9)
    df['SMA_21'] = ta.sma(df['close'], length=21)
    # Initialize Bollinger Bands Indicator
    
    
    return df


In [12]:
def plot_chart(symbol, df, theme):
    dark_palette = {}
    dark_palette["bg_color"] = "#2e2e2e"
    dark_palette["plot_bg_color"] = "#2e2e2e"
    dark_palette["grid_color"] = "#595656"
    dark_palette["text_color"] = "#ffffff"
    dark_palette["dark_candle"] = "#226287"
    dark_palette["light_candle"] = "#a6a4a4"
    dark_palette["volume_color"] = "#5c285b"
    dark_palette["border_color"] = "#ffffff"
    dark_palette["color_1"] = "#5c285b"
    dark_palette["color_2"] = "#802c62"
    dark_palette["color_3"] = "#a33262"
    dark_palette["color_4"] = "#c43d5c"
    dark_palette["color_5"] = "#de4f51"
    dark_palette["color_6"] = "#f26841"
    dark_palette["color_7"] = "#fd862b"
    dark_palette["color_8"] = "#ffa600"
    dark_palette["color_9"] = "#3366d6"
    light_palette = {}
    light_palette["bg_color"] = "#ffffff"
    light_palette["plot_bg_color"] = "#ffffff"
    light_palette["grid_color"] = "#e6e6e6"
    light_palette["text_color"] = "#2e2e2e"
    light_palette["dark_candle"] = "#4d98c4"
    light_palette["light_candle"] = "#b1b7ba"
    light_palette["volume_color"] = "#c74e96"
    light_palette["border_color"] = "#2e2e2e"
    light_palette["color_1"] = "#5c285b"
    light_palette["color_2"] = "#802c62"
    light_palette["color_3"] = "#a33262"
    light_palette["color_4"] = "#c43d5c"
    light_palette["color_5"] = "#de4f51"
    light_palette["color_6"] = "#f26841"
    light_palette["color_7"] = "#fd862b"
    light_palette["color_8"] = "#ffa600"
    light_palette["color_9"] = "#3366d6"
    palette = light_palette
    if theme == "Dark Mode":
        palette = dark_palette
    #  Create sub plots
    fig = make_subplots(rows=4, cols=1, subplot_titles=[f"{symbol} Chart", '', '', 'Volume'], \
                        specs=[[{"rowspan": 3, "secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}],
                               [{"secondary_y": True}]], \
                        vertical_spacing=0.04, shared_xaxes=True)
    #  Plot close price
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['Open'],
                                 close=df['Close'],
                                 low=df['Low'],
                                 high=df['High'],
                                 increasing_line_color=palette['light_candle'], decreasing_line_color=palette['dark_candle'], name='Price'), row=1, col=1)
    #  Add SMAs
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_9'], line=dict(color=palette['color_3'], width=1), name="SMA 9"),
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_21'], line=dict(color=palette['color_8'], width=1), name="SMA 21"), row=1,
                  col=1)
    
    #  Volume Histogram
    fig.add_trace(go.Bar(
        name='Volume',
        x=df.index, y=df['volume'], marker_color=palette['volume_color']), row=4,col=1)
    fig.update_layout(
            title={'text': '', 'x': 0.5},
            font=dict(family="Verdana", size=12, color=palette["text_color"]),
            autosize=True,
            width=1280, height=720,
            xaxis={"rangeslider": {"visible": False}},
            plot_bgcolor=palette["plot_bg_color"],
            paper_bgcolor=palette["bg_color"])
    fig.update_yaxes(visible=False, secondary_y=True)
    #  Change grid color
    fig.update_xaxes(showline=True, linewidth=1, linecolor=palette["grid_color"],gridcolor=palette["grid_color"])
    fig.update_yaxes(showline=True, linewidth=1, linecolor=palette["grid_color"],gridcolor=palette["grid_color"])
    #  Create output file
    #file_name = f"{symbol}_chart.png"
    #fig.write_image(file_name, format="png")
    return fig

In [13]:
symbol = "btcusdt"
df = load_data_2(symbol)
#  Run Dash server
app = Dash(__name__)
app.layout = html.Div([
    html.H2('Monoceros'),
    html.P("Select Theme:"),
    dcc.Dropdown(
        id="dropdown",
        options=['Dark Mode', 'Light Mode'],
        value='Dark Mode',
        clearable=False,
    ),
    dcc.Graph(id="graph"),
])
@app.callback(
    Output("graph", "figure"),
    Input("dropdown", "value"))
def display_color(theme):
    fig = plot_chart(symbol, df, theme)
    return fig
app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'